In [3]:
import numpy as np
import pandas as pd
import csv
from sklearn.feature_extraction import DictVectorizer
from scipy.io import loadmat

def cQ(X):
        returnAr = []
        key = np.array((0, 1))
        for i in range(X.shape[1]):
            returnAr.append(np.all(np.in1d(X[:, i], key)))
        return returnAr

In [400]:
############## Decision Tree Implementation ############
class Node:
    def __init__(self, j, beta, isLeaf, C, leftChild, rightChild):
        self.J = j
        self.beta = beta
        self.isLeaf = isLeaf
        self.C = C
        self.leftChild = leftChild
        self.rightChild = rightChild
    
    def predict(self, X):
        if self.isLeaf:
            return self.C
        elif X[self.J] < self.beta:
            return self.leftChild.predict(X)
        else:
            return self.rightChild.predict(X)

class Tree:
    def __init__(self, X, y, randomSubset, catQuant = None, max_depth = 30):
        self.y = y
        self.X = X
        if catQuant is None:
            self.catQuant = self.cQ(self.X)
        else:
            self.catQuant = catQuant
        self.subset = randomSubset
        self.max_depth = max_depth
        self.root = self.growTree(X, y, self.catQuant, self.subset, max_depth, 0)
        
    def growTree(self, X, y, catQuant, randomSubset, max_depth, curr_depth):
        if np.isnan(np.mean(y)):
            print('error A')
            return 
        if curr_depth == max_depth or np.mean(y) > 0.95 or np.mean(y) < 0.05:
            return Node(None, None, True, np.rint(np.mean(y)), None, None) #returns a leaf
        else:
            j, beta = self.findSplit(X, y, catQuant, randomSubset)
            SlX, SlY, SrX, SrY = self.split(X, y, j, beta)
            return Node(j, beta, False, np.rint(np.mean(y)), self.growTree(SlX, SlY, catQuant, randomSubset, max_depth, curr_depth + 1), self.growTree(SrX, SrY, catQuant, randomSubset, max_depth, curr_depth + 1))
    
    def findSplit(self, X, y, catQuant, randomSubset = False):
        minCost = None
        j = None
        beta = None
        subset = []
        if not randomSubset:
            subset = range(X.shape[1])
        else:
            subset = list(np.random.choice(X.shape[1], np.rint(np.sqrt(X.shape[1]), replace = False)))
        
        for i in subset:
            if catQuant[i]: # if categorical variable
                cost = self.J(X, y, i, 0.5)
                if minCost == None or cost < minCost:
                    j = i
                    beta = 0.5
                    minCost = cost
            else:
                sortedOrder = X[:,i].argsort()
                sortedX = X[sortedOrder, i]
                sortedY = y[sortedOrder]
                # Sorting complete. Now to initialize the O(n) search for the best split
                totalObs = X.shape[0]
                if np.isnan(np.mean(y)):
                    print('error B')
                    return
                total1 = np.mean(y) * totalObs
                total0 = totalObs - total1
                
                # Initializing
                if sortedY[0] == 1:
                    Lcat0 = 0
                    Lcat1 = 1
                    Rcat0 = total0
                    Rcat1 = total1 - 1
                else:
                    Lcat0 = 1
                    Lcat1 = 0
                    Rcat0 = total0 - 1
                    Rcat1 = total1
                    
                bestCostForThisCat = self.J_fast(Lcat0, Lcat1, Rcat0, Rcat1)
                bestSplitForThisCat = sortedX[0]

                for k in range(1, totalObs - 1):
                    if sortedY[k] == 1:
                        Lcat1 = Lcat1 + 1
                        Rcat1 = Rcat1 - 1
                    else:
                        Lcat0 = Lcat0 + 1
                        Rcat0 = Rcat0 - 1
                    testcost = self.J_fast(Lcat0, Lcat1, Rcat0, Rcat1)
                    if testcost < bestCostForThisCat:
                        bestCostForThisCat = testcost
                        bestSplitForThisCat = sortedX[k]
                
                if minCost == None or bestCostForThisCat < minCost:
                    j = i
                    beta = bestSplitForThisCat
                    minCost = bestCostForThisCat
        
#         print(j, beta)
#         _, sly, _, sry = (self.split(X, y, j, beta))
#         print(sly, sry)
        return j, beta

    
    def J(self, X, y, j, beta): # returns cost
        SlX, SlY, SrX, SrY = self.split(X, y, j, beta)
        a = SlX.shape[0] * self.H(SlY)
        b = SrX.shape[0] * self.H(SrY)
        c = SlX.shape[0] + SrX.shape[0]
        return (a + b) / c
    
    def H(self, y): # returns H
        if len(y) == 0:
            return 0
        p1 = np.mean(y)
        p0 = 1 - p1
        return -((p0 * np.log2(p0)) + (p1 * np.log2(p1)))
    
    def J_fast(self, l0, l1, r0, r1): # returns faster cost
        a = (l0 + l1) * self.H_fast(l0, l1)
        b = (r0 + r1) * self.H_fast(r0, r1)
        c = l0 + l1 + r0 + r1
        return (a + b) / c
    
    
    def H_fast(self, y0, y1):
        if y0 == 0 and y1 == 0:
            print ('error')
            return
        if y0 == 0:
            return -(1 * np.log2(1))
        if y1 == 0:
            return -(1 * np.log2(1))
            
        tot = y0 + y1
        p1 = y1/tot
        p0 = y0/tot
        return -((p0 * np.log2(p0)) + (p1 * np.log2(p1)))
    
    def cQ(self, X):
        returnAr = []
        key = np.array((0, 1))
        for i in range(X.shape[1]):
            returnAr.append(np.all(np.in1d(X[:, i], key)))
        return returnAr
    
    def split(self, X, y, j, beta): # Splits X into Sl and Sr based on j and beta
        SlX = []
        SlY = []
        SrX = []
        SrY = []
        for n in range(X.shape[0]):
            if X[n, j] <= beta:
                SlY.append(n)
                SlX.append(n)
            else:
                SrY.append(n)
                SrX.append(n)
        return X[SlX, :], y[SlY], X[SrX, :], y[SrY]
    
    def predict(self, X):
        return self.root.predict(X)
        
############ End Decision Trees #########

In [300]:
############ Start Random Forest ########
class Forest:
    def __init__(self, X, y, numTrees, max_depth):
        self.X = X
        self.y = y
        self.numTrees = numTrees
        self.catQuant = self.cQ(X)
        self.forest = self.growForest(X, y, self.numTrees, self.catQuant, max_depth)
        
    def growForest(self, X, y, numTrees, catQuant, max_depth): # Returns forest as list of trees
        forest = []
        for i in range(numTrees):
            subset = X[np.random.choice(X.shape[0], X.shape[0], replace = True),:]
            tree = Tree(subset, y, True, catQuant, max_depth[i])
            forest.append(tree)
        return forest
    
    def predict(self, X):
        prediction = []
        for i in range(numTrees):
            prediction.append(forest[i].predict(X))
        return np.rint(np.mean(prediction))
            
    def cQ(self, X):
        returnAr = []
        key = np.array((0, 1))
        for i in range(X.shape[1]):
            returnAr.append(np.all(np.in1d(X[:, i], key)))
        return returnAr
    
############ End Random Forest ##########

In [4]:
######### Preprocessing function ########

# Finds mode, returns as dict
def modeFinder(file):
    mode = {}
    df = pd.read_csv(file)
    for col in df:
        x = df[col]
        mode[col] = x.value_counts().index[0]
    return mode


In [13]:
# Census Cleaning

with open('hw5_census_dist/train_data.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader)
print(rows[0])

census_categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
census_continuous = ['fnlwgt', 'education-num', 'capital-gain', 'hours-per-week', 'age', 'capital-loss']
y = []

# Mode finder
mode = modeFinder('hw5_census_dist/train_data.csv')

for row in rows:
    for key, value in row.items():
        if value is "?":
            row[key] = mode[key]
    for label in census_continuous:
        row[label] = float(row[label])
    y.append(row.get('label'))
    del row['label']
    

vec = DictVectorizer(dtype = int)
X = vec.fit_transform(rows).toarray()
cols = vec.get_feature_names()
y = np.asmatrix(y)

census = np.array(np.concatenate((X, y.T), axis=1), dtype = int)
np.random.shuffle(census)
trainX = census[:28000, :105]
trainY = census[:28000, 105]
validX = census[28000:, :105]
validY = census[28000:, 105]

{'capital-loss': '0', 'age': '59', 'label': '0', 'capital-gain': '0', 'sex': 'Male', 'marital-status': 'Never-married', 'education-num': '5', 'hours-per-week': '50', 'race': 'Black', 'fnlwgt': '307423', 'occupation': 'Other-service', 'workclass': 'Private', 'relationship': 'Not-in-family', 'education': '9th', 'native-country': 'United-States'}


In [16]:
type(rows[0])

dict

In [401]:
# Census q4

# Trying various tree max_depths
max_depths = [5, 10, 15, 20, 25, 30, 35, 40]
train_error = []
valid_error = []

for i in max_depths:
    terr = 0
    verr = 0
    tree = Tree(trainX, trainY, False, max_depth = i)
    for i in range(trainX.shape[0]):
        terr = terr + np.sum(np.square(tree.predict(trainX[i]) - trainY[i]))
    print('terr')
    print(terr)
    train_error.append(terr)
    for i in range(validX.shape[0]):
        verr = np.sum(np.square(tree.predict(validX[i]) - validY[i]))
    valid_error.append(verr)


terr
4382.0


/Users/Paul/anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A
error A


AttributeError: 'NoneType' object has no attribute 'predict'

In [316]:
train_error

[]

In [196]:
### Spam
spam = loadmat("dist/spam_data.mat")
spam_train_data = spam.get('training_data')
spam_train_labels = spam.get('training_labels')
spam_test_data = spam.get('test_data')






In [213]:
# Titanic
with open('hw5_titanic_dist/titanic_training.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

titanic_continuous = ['age', 'fare', 'parch', 'pclass', 'sibsp']
y = []
mode = modeFinder('hw5_titanic_dist/titanic_training.csv')

for row in rows:
    for key, value in row.items():
        if value is "":
            row[key] = mode[key]
    for label in titanic_continuous:
        row[label] = float(row[label])
    y.append(row.get('survived'))
    del row['survived']

vec = DictVectorizer(dtype = int)
X = vec.fit_transform(rows).toarray()
cols = vec.get_feature_names()
# may need to convert y to int
X.shape

X.train = 

(1000, 904)

# 3. 
A. I used dictVectorizer to convert categorical features into one-hot dummies. To deal with missing values, I imputed with the mode.

B. As stopping criteria, I set a max depth that the tree could grow or stopped if the tree only contained labels of one category. 

C. Yes, I created a boolean array called catQuant that is passed through the growTree function that informs the function whether each variable is categorical or quantitative. Thus I only need to detect categorical vs quantitative variables once at the start of training. 

D. Random forests were implemented by first adding a boolean variable randomSubset to methods in the Tree class, which, if true, instructs the split finding function of growTree to only use a subset of columns for splitting. Then, a simple wrapper was added that grows trees on subsets of the data and stores the forest as a list of trees. 

E. I think the modefinder is pretty cool! It is an imputation helper that reads the data and finds the mode values for each column, which are returned as a dictionary{variable:modevalue}. This lets me use my existing dictionary-based data preprocessing framework to also easily impute values. 

In [212]:
x = np.array([0, 1, 2, 3])


SyntaxError: invalid syntax (<ipython-input-212-ab894d1e86a3>, line 3)

In [385]:
0 == 0.

True